<a href="https://colab.research.google.com/github/AlexisStdp/AlexisStdp.github.io/blob/master/Datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sklearn
import numpy as np
import tensorflow as tf
import pandas as pd
import xgboost as xgb

In [2]:
from tensorflow import keras
from tensorflow.keras import layers, regularizers, Sequential, Model, backend as K
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

In [3]:
X_train = pd.read_csv('X_train.csv')
X_train

,Id,Date,Lat,long,PropertyTy,Area,postCode,City,Neighborhood,PropertyTy.1,transYear,transMonth,transDays,transQtr,Rooms,Baths,Parking,HasFirepla,HasPool,HasGarage,HasAirCond
0,0,15.12.2010,-37.829457,145.130499,Unit,193,3128,Box Hill,Whitehorse (C) - Box Hill,Unit,2010,7,198,3,3.0,2.0,NaN,NaN,NaN,NaN,0.0
1,1,30.06.2010,-37.901293,145.040301,Unit,1222,3204,Ormond,Glen Eira (C) - Caulfield,Unit,2010,1,30,1,2.0,NaN,1.0,NaN,0.0,0.0,NaN
2,2,14.06.2011,-38.088126,145.488687,House,528,3810,Pakenham,Cardinia (S) - Pakenham,House,2011,13,379,5,4.0,2.0,2.0,0.0,0.0,NaN,0.0
3,3,12.05.2011,-37.885781,144.776041,House,557,3028,Altona Meadows,Hobsons Bay (C) - Altona,House,2011,12,346,4,NaN,3.0,4.0,NaN,0.0,1.0,0.0
4,4,21.09.2010,-37.836286,144.943116,House,111,3207,Port Melbourne,Port Phillip (C) - West,House,2010,4,113,2,NaN,1.0,NaN,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84563,84563,29.12.2012,-37.811613,145.123794,Unit,138,3129,Box Hill North,Whitehorse (C) - Box Hill,Unit,2012,31,943,11,NaN,NaN,NaN,0.0,NaN,0.0,0.0
84564,84564,26.02.2011,-37.830895,145.044432,House,249,3123,Hawthorn East,Boroondara (C) - Hawthorn,House,2011,9,271,3,3.0,1.0,1.0,1.0,0.0,0.0,0.0
84565,84565,02.06.2010,-37.823592,145.244889,House,670,3135,Heathmont,Maroondah (C) - Ringwood,House,2010,1,2,1,NaN,1.0,2.0,NaN,NaN,1.0,1.0
84566,84566,03.10.2011,-37.818219,145.242537,Unit,752,3135,Ringwood East,Maroondah (C) - Ringwood,Unit,2011,17,490,6,2.0,NaN,NaN,0.0,0.0,0.0,0.0


In [4]:
X_train['Day_of_month'] = X_train['Date']
X_train['Month'] = X_train['Date']
X_train['Year'] = X_train['Date']

In [5]:
X_train.columns

Index(['Id', 'Date', 'Lat', 'long', 'PropertyTy', 'Area', 'postCode', 'City',
       'Neighborhood ', 'PropertyTy.1', 'transYear', 'transMonth', 'transDays',
       'transQtr', 'Rooms', 'Baths', 'Parking', 'HasFirepla', 'HasPool',
       'HasGarage', 'HasAirCond', 'Day_of_month', 'Month', 'Year'],
      dtype='object')

In [33]:
len(X_train['Date'])

84568

In [7]:
# print(X_train['Date'][0].split('.'))

for i,_ in enumerate(X_train['Date']):
  temp = X_train['Date'][i].split('.')
  X_train['Day_of_month'][i] = int(temp[0])
  X_train['Month'][i] = int(temp[1])
  X_train['Year'][i] = int(temp[2])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [37]:
X_train_ready['Day_of_month'] = pd.to_numeric(X_train_ready['Day_of_month'])
X_train_ready['Month'] = pd.to_numeric(X_train_ready['Month'])
X_train_ready['Year'] = pd.to_numeric(X_train_ready['Year'])

In [8]:
enc = sklearn.preprocessing.LabelEncoder()
enc.fit(X_train['City'])
X_train['City_encoded'] = enc.transform(X_train['City'])

In [9]:
enc1 = sklearn.preprocessing.LabelEncoder()
enc1.fit(X_train['PropertyTy'])
X_train['PropertyTy_encoded'] = enc1.transform(X_train['PropertyTy'])

In [10]:
X_train.drop('PropertyTy.1', axis=1, inplace=True)

In [11]:
enc3 = sklearn.preprocessing.LabelEncoder()
enc3.fit(X_train['Neighborhood '])
X_train['Neighborhood_encoded'] = enc3.transform(X_train['Neighborhood '])

In [12]:
X_train

,Id,Date,Lat,long,PropertyTy,Area,postCode,City,Neighborhood,transYear,transMonth,transDays,transQtr,Rooms,Baths,Parking,HasFirepla,HasPool,HasGarage,HasAirCond,Day_of_month,Month,Year,City_encoded,PropertyTy_encoded,Neighborhood_encoded
0,0,15.12.2010,-37.829457,145.130499,Unit,193,3128,Box Hill,Whitehorse (C) - Box Hill,2010,7,198,3,3.0,2.0,NaN,NaN,NaN,NaN,0.0,15,12,2010,43,1,64
1,1,30.06.2010,-37.901293,145.040301,Unit,1222,3204,Ormond,Glen Eira (C) - Caulfield,2010,1,30,1,2.0,NaN,1.0,NaN,0.0,0.0,NaN,30,6,2010,278,1,20
2,2,14.06.2011,-38.088126,145.488687,House,528,3810,Pakenham,Cardinia (S) - Pakenham,2011,13,379,5,4.0,2.0,2.0,0.0,0.0,NaN,0.0,14,6,2011,279,0,11
3,3,12.05.2011,-37.885781,144.776041,House,557,3028,Altona Meadows,Hobsons Bay (C) - Altona,2011,12,346,4,NaN,3.0,4.0,NaN,0.0,1.0,0.0,12,5,2011,8,0,24
4,4,21.09.2010,-37.836286,144.943116,House,111,3207,Port Melbourne,Port Phillip (C) - West,2010,4,113,2,NaN,1.0,NaN,0.0,0.0,0.0,NaN,21,9,2010,289,0,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84563,84563,29.12.2012,-37.811613,145.123794,Unit,138,3129,Box Hill North,Whitehorse (C) - Box Hill,2012,31,943,11,NaN,NaN,NaN,0.0,NaN,0.0,0.0,29,12,2012,44,1,64
84564,84564,26.02.2011,-37.830895,145.044432,House,249,3123,Hawthorn East,Boroondara (C) - Hawthorn,2011,9,271,3,3.0,1.0,1.0,1.0,0.0,0.0,0.0,26,2,2011,175,0,6
84565,84565,02.06.2010,-37.823592,145.244889,House,670,3135,Heathmont,Maroondah (C) - Ringwood,2010,1,2,1,NaN,1.0,2.0,NaN,NaN,1.0,1.0,2,6,2010,177,0,40
84566,84566,03.10.2011,-37.818219,145.242537,Unit,752,3135,Ringwood East,Maroondah (C) - Ringwood,2011,17,490,6,2.0,NaN,NaN,0.0,0.0,0.0,0.0,3,10,2011,298,1,40


In [13]:
y_train = pd.read_csv('y_train.csv')
y_train.drop('Id', axis=1, inplace=True)
y_train

,Price
0,500000
1,540000
2,365000
3,608000
4,787000
...,...
84563,430000
84564,826000
84565,505000
84566,410000


In [14]:
X_train_ready = X_train.copy()

In [15]:
X_train_ready.drop(['Id','Date', 'PropertyTy', 'City', 'Neighborhood '], axis=1, inplace=True)

In [16]:
X_train_ready['Weekday'] = X_train_ready['transDays'].apply(lambda x: x % 7)

In [17]:
X_train_ready['Quarter'] = X_train_ready['transQtr'].apply(lambda x: (x + 1) % 4) ## season

In [18]:
X_train_ready

,Lat,long,Area,postCode,transYear,transMonth,transDays,transQtr,Rooms,Baths,Parking,HasFirepla,HasPool,HasGarage,HasAirCond,Day_of_month,Month,Year,City_encoded,PropertyTy_encoded,Neighborhood_encoded,Weekday,Quarter
0,-37.829457,145.130499,193,3128,2010,7,198,3,3.0,2.0,NaN,NaN,NaN,NaN,0.0,15,12,2010,43,1,64,2,0
1,-37.901293,145.040301,1222,3204,2010,1,30,1,2.0,NaN,1.0,NaN,0.0,0.0,NaN,30,6,2010,278,1,20,2,2
2,-38.088126,145.488687,528,3810,2011,13,379,5,4.0,2.0,2.0,0.0,0.0,NaN,0.0,14,6,2011,279,0,11,1,2
3,-37.885781,144.776041,557,3028,2011,12,346,4,NaN,3.0,4.0,NaN,0.0,1.0,0.0,12,5,2011,8,0,24,3,1
4,-37.836286,144.943116,111,3207,2010,4,113,2,NaN,1.0,NaN,0.0,0.0,0.0,NaN,21,9,2010,289,0,61,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84563,-37.811613,145.123794,138,3129,2012,31,943,11,NaN,NaN,NaN,0.0,NaN,0.0,0.0,29,12,2012,44,1,64,5,0
84564,-37.830895,145.044432,249,3123,2011,9,271,3,3.0,1.0,1.0,1.0,0.0,0.0,0.0,26,2,2011,175,0,6,5,0
84565,-37.823592,145.244889,670,3135,2010,1,2,1,NaN,1.0,2.0,NaN,NaN,1.0,1.0,2,6,2010,177,0,40,2,2
84566,-37.818219,145.242537,752,3135,2011,17,490,6,2.0,NaN,NaN,0.0,0.0,0.0,0.0,3,10,2011,298,1,40,0,3


In [29]:
for i,j in enumerate(X_train_ready['Day_of_month'].to_numpy()):
  if type(j) is not int:
    print(i)
for i,j in enumerate(X_train_ready['Month'].to_numpy()):
  if type(j) is not int:
    print(i)
for i,j in enumerate(X_train_ready['Year'].to_numpy()):
  if type(j) is not int:
    print(i)

In [38]:
leave_out = 10000
xgb_model = xgb.XGBRegressor(random_state=42, max_depth=10, objective='reg:squarederror')
xgb_model.fit(X_train_ready[:-leave_out], y_train[:-leave_out])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [39]:
print(sklearn.metrics.mean_squared_error(xgb_model.predict(X_train_ready[:-leave_out]), y_train[:-leave_out], squared=False))
print(sklearn.metrics.mean_squared_error(xgb_model.predict(X_train_ready[leave_out:]), y_train[leave_out:], squared=False))

105133.48460387246
115781.4189729369


In [70]:
import lightgbm as lgb
params = {'num_leaves': 100}
lgb_data = lgb.Dataset(X_train_ready, label=y_train)
bst = lgb.train(params, lgb_data, 100)
# lgb.cv(param, train_data, num_round, nfold=5)

In [71]:
print(sklearn.metrics.mean_squared_error(bst.predict(X_train_maybe[:-leave_out]), y_train[:-leave_out], squared=False))
print(sklearn.metrics.mean_squared_error(bst.predict(X_train_maybe[leave_out:]), y_train[leave_out:], squared=False))

190694.14156901374
189552.51834678216


Without NaN

In [61]:
X_train_maybe = X_train_ready.copy()

In [62]:
X_train_maybe['HasPool'].fillna(0, inplace=True)
X_train_maybe['HasGarage'].fillna(0, inplace=True)
X_train_maybe['HasFirepla'].fillna(0, inplace=True)
X_train_maybe['HasAirCond'].fillna(0, inplace=True)

In [63]:
X_train_maybe['Parking'].fillna(0, inplace=True)
X_train_maybe['Baths'].fillna(0, inplace=True)
X_train_maybe['Rooms'].fillna(0, inplace=True)

In [72]:
X_train_maybe

,Lat,long,Area,postCode,transYear,transMonth,transDays,transQtr,Rooms,Baths,Parking,HasFirepla,HasPool,HasGarage,HasAirCond,Day_of_month,Month,Year,City_encoded,PropertyTy_encoded,Neighborhood_encoded,Weekday,Quarter
0,-37.829457,145.130499,193,3128,2010,7,198,3,3.0,2.0,0.0,0.0,0.0,0.0,0.0,15,12,2010,43,1,64,2,0
1,-37.901293,145.040301,1222,3204,2010,1,30,1,2.0,0.0,1.0,0.0,0.0,0.0,0.0,30,6,2010,278,1,20,2,2
2,-38.088126,145.488687,528,3810,2011,13,379,5,4.0,2.0,2.0,0.0,0.0,0.0,0.0,14,6,2011,279,0,11,1,2
3,-37.885781,144.776041,557,3028,2011,12,346,4,0.0,3.0,4.0,0.0,0.0,1.0,0.0,12,5,2011,8,0,24,3,1
4,-37.836286,144.943116,111,3207,2010,4,113,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,21,9,2010,289,0,61,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84563,-37.811613,145.123794,138,3129,2012,31,943,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,12,2012,44,1,64,5,0
84564,-37.830895,145.044432,249,3123,2011,9,271,3,3.0,1.0,1.0,1.0,0.0,0.0,0.0,26,2,2011,175,0,6,5,0
84565,-37.823592,145.244889,670,3135,2010,1,2,1,0.0,1.0,2.0,0.0,0.0,1.0,1.0,2,6,2010,177,0,40,2,2
84566,-37.818219,145.242537,752,3135,2011,17,490,6,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3,10,2011,298,1,40,0,3


In [108]:
from sklearn.ensemble import GradientBoostingRegressor
sk_gbm = GradientBoostingRegressor(random_state=42, max_depth=10)
sk_gbm.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=10,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [109]:
print(sklearn.metrics.mean_squared_error(sk_gbm.predict(X_train_maybe[:-leave_out]), y_train[:-leave_out], squared=False))
print(sklearn.metrics.mean_squared_error(sk_gbm.predict(X_train_maybe[leave_out:]), y_train[leave_out:], squared=False))

95258.26056886115
108502.01112492318


In [76]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=10, random_state=42)
regr.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [77]:
print(sklearn.metrics.mean_squared_error(regr.predict(X_train_maybe[:-leave_out]), y_train[:-leave_out], squared=False))
print(sklearn.metrics.mean_squared_error(regr.predict(X_train_maybe[leave_out:]), y_train[leave_out:], squared=False))

200626.8790145207
201542.45794793376


In [84]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [85]:
print(sklearn.metrics.mean_squared_error(lr.predict(X_train_maybe[:-leave_out]), y_train[:-leave_out], squared=False))
print(sklearn.metrics.mean_squared_error(lr.predict(X_train_maybe[leave_out:]), y_train[leave_out:], squared=False))

355209.5890155941
352528.93050029676


In [88]:
from sklearn.svm import SVR
svr = SVR(max_iter=1000, kernel='rbf')
svr.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=1000, shrinking=True, tol=0.001, verbose=False)

In [89]:
print(sklearn.metrics.mean_squared_error(svr.predict(X_train_maybe[:-leave_out]), y_train[:-leave_out], squared=False))
print(sklearn.metrics.mean_squared_error(svr.predict(X_train_maybe[leave_out:]), y_train[leave_out:], squared=False))

629001.8326015454
627749.1708672319


In [94]:
from sklearn.svm import SVR
svr = SVR(max_iter=1000, kernel='poly')
svr.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='poly', max_iter=1000, shrinking=True, tol=0.001, verbose=False)

In [95]:
print(sklearn.metrics.mean_squared_error(svr.predict(X_train_maybe[:-leave_out]), y_train[:-leave_out], squared=False))
print(sklearn.metrics.mean_squared_error(svr.predict(X_train_maybe[leave_out:]), y_train[leave_out:], squared=False))

628964.8035077819
627711.9862902322


In [92]:
from sklearn.linear_model import ElasticNet
regr = ElasticNet(random_state=42, max_iter=1000)
regr.fit(X_train_maybe, y_train)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=10000, normalize=False, positive=False, precompute=False,
           random_state=42, selection='cyclic', tol=0.0001, warm_start=False)

In [93]:
print(sklearn.metrics.mean_squared_error(regr.predict(X_train_maybe[:-leave_out]), y_train[:-leave_out], squared=False))
print(sklearn.metrics.mean_squared_error(regr.predict(X_train_maybe[leave_out:]), y_train[leave_out:], squared=False))

362781.5967797893
360125.25225522637


Let's start stacking

In [98]:
xgb_model = xgb.XGBRegressor(random_state=42, max_depth=10, objective='reg:squarederror')

params = {'num_leaves': 100}
lgb_data = lgb.Dataset(X_train_ready, label=y_train)
bst = lgb.train(params, lgb_data, 100)

from sklearn.ensemble import GradientBoostingRegressor
sk_gbm = GradientBoostingRegressor(random_state=42, max_depth=10)

# from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=10, random_state=42)

# from sklearn.linear_model import LinearRegression
lr = LinearRegression()

# from sklearn.svm import SVR
svr = SVR(max_iter=1000, kernel='rbf')
svr = SVR(max_iter=1000, kernel='poly')

# from sklearn.linear_model import ElasticNet
regr = ElasticNet(random_state=42, max_iter=1000)

In [225]:
leave_out = 40000

In [105]:
# XGBOOST MODELS
xgb_model = xgb.XGBRegressor(random_state=42, max_depth=10, objective='reg:squarederror', n_estimators=100)
xgb_model.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])

xgb_model1 = xgb.XGBRegressor(random_state=42, max_depth=10, objective='reg:squarederror', learning_rate=0.05, n_estimators=100)
xgb_model1.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])

xgb_model1 = xgb.XGBRegressor(random_state=42, max_depth=10, objective='reg:squarederror', learning_rate=0.05, n_estimators=100)
xgb_model1.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [113]:
# LightGBM MODELS - FAST FAST !!
bst = lgb.train({'num_leaves': 100}, lgb.Dataset(X_train_ready[:-leave_out], label=y_train[:-leave_out]), num_boost_round=100)
bst1 = lgb.train({'num_leaves': 100}, lgb.Dataset(X_train_ready[:-leave_out], label=y_train[:-leave_out]), num_boost_round=200)
bst2 = lgb.train({'num_leaves': 100}, lgb.Dataset(X_train_ready[:-leave_out], label=y_train[:-leave_out]), num_boost_round=300)
bst3 = lgb.train({'num_leaves': 100}, lgb.Dataset(X_train_ready[:-leave_out], label=y_train[:-leave_out]), num_boost_round=400)
bst4 = lgb.train({'num_leaves': 100}, lgb.Dataset(X_train_ready[:-leave_out], label=y_train[:-leave_out]), num_boost_round=500)

In [115]:
# sklearn GB-reg
sk_gbm = GradientBoostingRegressor(random_state=42, max_depth=10)
sk_gbm.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])
sk_gbm1 = GradientBoostingRegressor(random_state=42, max_depth=11)
sk_gbm1.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])
sk_gbm1 = GradientBoostingRegressor(random_state=42, max_depth=11, n_estimators=100)
sk_gbm1.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=11,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [120]:
# sklearn RF-reg
regr = RandomForestRegressor(max_depth=10, random_state=42, n_estimators=100)
regr.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])
regr1 = RandomForestRegressor(max_depth=10, random_state=42, n_estimators=150)
regr1.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__mai

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [126]:
# sklearn LR
lr = LinearRegression()
lr.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [129]:
# sklearn ElasticNet
enet = ElasticNet(random_state=42, max_iter=1000, alpha=.8)
enet.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])
enet1 = ElasticNet(random_state=42, max_iter=1000, alpha=.9, l1_ratio=.9)
enet1.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])
enet2 = ElasticNet(random_state=42, max_iter=1000)
enet2.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4302156120694931.0, tolerance: 1102078172617.1133
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4784674329586068.0, tolerance: 1102078172617.1133
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3059636918434583.0, tolerance: 1102078172617.1133
  positive)


ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=42, selection='cyclic', tol=0.0001, warm_start=False)

In [134]:
# sklearn SVR
svr = SVR(max_iter=1000, kernel='rbf')
svr.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])
svr1 = SVR(max_iter=1000, kernel='poly')
svr1.fit(X_train_maybe[:-leave_out], y_train[:-leave_out])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='poly', max_iter=1000, shrinking=True, tol=0.001, verbose=False)

In [174]:
ensemble = [xgb_model, xgb_model1, bst, bst1, bst2, bst3, bst4, sk_gbm, sk_gbm1, \
            regr, regr1,# regr2, regr3, \
            lr, enet, enet1, enet2, svr, svr1]

In [187]:
y_stack = []
for m in ensemble:
  y_stack.append(m.predict(X_train_maybe[-leave_out:-5000]))

In [189]:
y_stack[-6] = y_stack[-6].reshape(5000,)

In [190]:
y_stack = np.array(y_stack)

In [191]:
y_stack = y_stack.T
y_stack.shape

(5000, 17)

In [226]:
king_gb = GradientBoostingRegressor(random_state=42, max_depth=3, n_estimators=10)
king_gb.fit(y_stack, y_train[-leave_out:-5000])
# king_lr = LinearRegression()
# king_lr.fit(y_stack, y_train[-leave_out:-5000])

ValueError: ignored

In [216]:
y_stack_val = []
for m in ensemble:
  y_stack_val.append(m.predict(X_train_maybe[-5000:]))

In [217]:
y_stack_val[-6] = y_stack_val[-6].reshape(5000,)

In [218]:
y_stack_val = np.array(y_stack_val).T

In [219]:
y_stack_val.shape

(5000, 17)

In [222]:
print(sklearn.metrics.mean_squared_error(king_gb.predict(y_stack), y_train[-leave_out:-5000], squared=False))
print(sklearn.metrics.mean_squared_error(king_gb.predict(y_stack_val), y_train[-5000:], squared=False))

202445.60757200775
212909.4560297642


In [224]:
print(sklearn.metrics.mean_squared_error(xgb_model.predict(X_train_ready[-5000:]), y_train[-leave_out:-5000], squared=False))
print(sklearn.metrics.mean_squared_error(xgb_model.predict(X_train_ready[-5000:]), y_train[-5000:], squared=False))

512703.785837011
172748.13273596848


NN

In [82]:
def nn(input_dim, output_dim, hidden_layers = 3, nodes = 100, l2_regularization = 0.0,
       s = 0.05, random = False, learning_rate = 0.001):
  input_layer = Input(shape=(input_dim, ), name='Input_Layer')
  l = Dense(nodes, activation = 'relu', 
            kernel_initializer = tf.compat.v1.keras.initializers.RandomUniform(minval=-s, maxval=s),
            kernel_regularizer = regularizers.l2(l2_regularization),
            bias_regularizer = regularizers.l2(l2_regularization), 
            trainable=not(random), name=f'ReLU_Layer_{0}')(input_layer)
  for i in range(hidden_layers - 1):
    l = Dense(nodes, activation = 'relu', 
            kernel_initializer = tf.compat.v1.keras.initializers.RandomUniform(minval=-s, maxval=s),
            kernel_regularizer = regularizers.l2(l2_regularization),
            bias_regularizer = regularizers.l2(l2_regularization), 
            trainable=not(random), name=f'ReLU_Layer_{i+1}')(l)
  l = Dense(output_dim, activation='linear',
            kernel_initializer = tf.compat.v1.keras.initializers.RandomUniform(minval=-s, maxval=s),
            bias_initializer = tf.compat.v1.keras.initializers.RandomUniform(minval=-s, maxval=s), 
            kernel_regularizer = regularizers.l2(l2_regularization),
            name = f"Output_Layer")(l)
  model = Model(inputs=[input_layer], outputs=l)
  loss_fct = 'MSE'
  model.compile(optimizer = tf.keras.optimizers.SGD(lr = learning_rate),
                loss = loss_fct, metrics = ['MSE'])
  return model

def nn_(nodes = 10):
  model = keras.Sequential()
  model.add(layers.Dense(nodes, activation="relu"))
  model.add(layers.Dense(nodes, activation="relu"))
  model.add(layers.Dense(1))
  model.compile(tf.keras.optimizers.Adam(0.01), 'MSE')
  return model

In [58]:
nn_x_train = X_train_maybe.to_numpy()

In [ ]:
# nn_model = nn(24, 1, learning_rate=0.001)
# nn_model = nn_(100)
nn_model.fit(np.asarray(X_train_maybe[:-leave_out]).astype('float32'),
             np.asarray(y_train[:-leave_out]).astype('float32'),
             batch_size=74568, epochs=10)

In [88]:
print(sklearn.metrics.mean_squared_error(nn_model.predict(np.asarray(nn_x_train[:-leave_out]).astype('float32')),
                                         y_train[:-leave_out], squared=False))
print(sklearn.metrics.mean_squared_error(nn_model.predict(np.asarray(nn_x_train[-leave_out:]).astype('float32')),
                                         y_train[-leave_out:], squared=False))

374533.2869509775
374679.1283184782
